# Preprocess the finetuning results

### Import libraries and install the embedding model

In [ ]:
!pip install sentence-transformers

In [ ]:
import os
import re
import json
import pandas as pd
import csv
import numpy as np
from collections import defaultdict, Counter
from sentence_transformers import SentenceTransformer, util

### File paths to preprocess

In [ ]:
files_to_format = [
    "finetuning_results/finetuning_results.txt",
    "finetuning_results/finetuning_results2.txt",
    "finetuning_results/finetuning_results3.txt",
    "finetuning_results/finetuning_results4.txt",
    "finetuning_results/finetuning_results5.txt",
    "finetuning_results/finetuning_results6.txt",
    "finetuning_results/finetuning_results7.txt",
    "finetuning_results/finetuning_results8.txt",
    "finetuning_results/finetuning_results9.txt",
    "finetuning_results/finetuning_results10.txt"
]

In [1]:
# Path to the CSV file
file_path = 'All_data.csv'

# Load the data with the detected delimiter
data = pd.read_csv(file_path, delimiter=";")

# Get only test set data
first_25_items = data.head(25)
additional_23_items = data.iloc[50:73]

combined_data = pd.concat([first_25_items, additional_23_items])

combined_data.to_csv('Test_finetuning_data.csv', index=False, sep=';')


### Get the true label for fine-tuning for evaluation

In [3]:
# Define the mapping of labels to column names
label_to_column = {
    '1': 'Municipality and residents engagement in the energy sector',
    '2': 'Energy storage and supplying energy in The Netherlands',
    '3': 'Wind and solar energy',
    '4': 'Market Determination Dynamics',
    '5': 'Landscapes and windmills tourism',
    '6': 'Hydrogen energy pipeline networks'
}

# Assuming 'combined_data' is your initial DataFrame and it has been loaded correctly
# Initialize a dictionary to store text to index mapping
text_to_index = {}
for idx, row in combined_data.iterrows():
    text_to_index[row['english'].strip()] = idx  # Map clean text to their row index

# Initialize all topic columns to '0'
for column in label_to_column.values():
    combined_data[column] = '0'

# Read the JSONL file and process each line
output_test_data = "content/validation.jsonl"

with open(output_test_data, 'r') as file:
    for line in file:
        entry = json.loads(line)

        text_line = entry['prompt']
        text_start = text_line.find("Text:") + 6  # Start after "Text:" plus space
        text_end = text_line.find("Topics:")
        # Extract the text and strip out the trailing new lines and extra spaces
        text = text_line[text_start:text_end].strip().rstrip("\\n")

        topic_line = entry['response']
        start = topic_line.find("Topics':") + 8
        end = topic_line.rfind("}")
        topics_list = topic_line[start:end].strip()
        topics = topics_list.split(", ") if topics_list else []

        # Update DataFrame based on extracted topics
        if text in text_to_index:
            idx = text_to_index[text]
            for label, column_name in label_to_column.items():
                if column_name in topics:
                    combined_data.at[idx, column_name] = '1'

# Save the filled CSV file
filled_csv_path_final = "content/test_set.csv"
combined_data.to_csv(filled_csv_path_final, index=False, sep=';')


# Print path to saved file
print(filled_csv_path_final)


Residents are needed. It's also a lot of fun to get involved. However, there are quick choices of principle that exclude other solutions. Very valuable on a small scale (including mienskip) and that certainly in combination with climate and reuse. Local initiatives can also lead to fragmentation while electricity must always be available, and inefficiencies.
I assign points to both "The municipality takes the lead and unburdens" and to "Residents do it themselves". This seems contradictory, and it is. Nevertheless, I also think the residents' own initiative is important. For support, for social cohesion, but also for management. A local cooperative will be able to supply energy significantly cheaper than a commercial party. I have a hesitation about sustainability. People move, neighborhoods change, engagement can become less. To what extent can a cooperative guarantee that everyone always has access to energy?
Given the installation of wind turbines in IJsselmeer, it would increase th

In [5]:
def format_finetuning_results(file_path, output_dir):
    """
    Format the fine-tuning results for preprocessing

    Args:
        file_path (str): file path to format
        output_dir (str): output directory in which the formatted files are saved
    """
    with open(file_path, 'r', encoding='ISO-8859-1') as file:
        for line in file:
            try:
                entry = json.loads(line)  # Try to parse each line as a JSON object

                text_line = entry['prompt']
                text_start = text_line.find("Text:") + 6
                text_end = text_line.find("Topics:")
                text = text_line[text_start:text_end].strip().rstrip("\\n")

                response = entry["response"]
                pattern = r"\[/INST\]\s*\{.*?\}\s*"
                # Search for the pattern in the input string
                match = re.search(pattern, response)
                topics_found = match.group(0)
                pattern2 = r"\{.*\}"
                match2 = re.search(pattern2, topics_found)
                topics = match2.group(0)

                base_name = os.path.splitext(os.path.basename(file_path))[0]
                output_path = os.path.join(output_dir, f'{base_name}_formatted.txt')
                result = "Text: "+ text + '\n' + topics + '\n\n'
                with open(output_path, 'a', encoding='utf-8') as output_file:
                    print(result)
                    output_file.write(result)
            except json.JSONDecodeError as e:
                print(f"Failed to decode JSON from line in file {file_path}: {str(e)}")


In [6]:
for file in files_to_format:
    format_finetuning_results(file, "finetuning_results")

Text: Residents are needed. It's also a lot of fun to get involved. However, there are quick choices of principle that exclude other solutions. Very valuable on a small scale (including mienskip) and that certainly in combination with climate and reuse. Local initiatives can also lead to fragmentation while electricity must always be available, and inefficiencies.
{'Topics': Municipality and residents engagement in the energy sector}


Text: I assign points to both "The municipality takes the lead and unburdens" and to "Residents do it themselves". This seems contradictory, and it is. Nevertheless, I also think the residents' own initiative is important. For support, for social cohesion, but also for management. A local cooperative will be able to supply energy significantly cheaper than a commercial party. I have a hesitation about sustainability. People move, neighborhoods change, engagement can become less. To what extent can a cooperative guarantee that everyone always has access t

In [9]:
# Formatted file paths
file_paths_to_process = [
    f"{fp.replace('.txt', '_formatted.txt')}"
    for fp in files_to_format
]

In [10]:
# Mapping dictionary for topics
topic_mapping = {
    "municipality and residents engagement in the energy sector": 1,
    "residents engagement": 1,
    "municipality engagement": 1,
    "residents engagement in the energy sector": 1,
    "municipality engagement in the energy sector": 1,
    "energy storage and supplying energy in the netherlands": 2,
    "energy storage and supply in the netherlands": 2,
    "energy storage in the netherlands": 2,
    "supplying energy in the netherlands": 2,
    "energy storage": 2,
    "wind and solar energy": 3,
    "solar energy": 3,
    "wind energy": 3,
    "market determination dynamics": 4,
    "market dynamics": 4,
    "market determination": 4,
    "landscapes and windmills tourism": 5,
    "landscapes and windmills": 5,
    "hydrogen energy pipeline networks": 6,
    "hydrogen energy pipeline": 6
}

# Load the Sentence Transformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Pre-compute embeddings for topic mapping keys
topic_embeddings = {topic: model.encode(topic, convert_to_tensor=True) for topic in topic_mapping.keys()}

def is_valid_topic(topic):
    """
    Check if a topic string is valid (not empty) when removing special characters

    Args:
        topic (str): the topic string
    Returns:
        bool : True if topic is valid, False otherwise

    """
    # Remove special characters and check if the resulting string is empty
    return re.sub(r'[^A-Za-z0-9]+', '', topic).strip() != ''

def process_topics(topics):
    """
    Map the topic list to the original list of topics
    The mapping is done using Sentence-Transformers embedding model (if the topic is not in 'topic_mapping')

    Args:
        topics (list): the list of topics to be preprocessed
    Returns:
        topic_numbers (list) : list of indices indicating the mapping of the topics to the original list

    """
    # Extract numbers and topic strings from the topics
    topic_numbers = set()
    threshold = 0.70  # Cosine similarity threshold for similarity

    if isinstance(topics, int):
        topic_numbers.add(topics)
    elif isinstance(topics, list):
        for topic in topics:
            if isinstance(topic, int):
                topic_numbers.add(topic)
            elif topic.isdigit():
                topic_numbers.add(int(topic))
            else:
                topic_lower = topic.lower().strip()  # Convert to lowercase
                if is_valid_topic(topic_lower):
                    topic_embedding = model.encode(topic_lower, convert_to_tensor=True)
                    # Perform cosine similarity matching
                    similarities = {key: util.pytorch_cos_sim(topic_embedding, emb).item() for key, emb in topic_embeddings.items()}
                    matched_topic, score = max(similarities.items(), key=lambda x: x[1])
                    if score >= threshold:
                        topic_numbers.add(topic_mapping[matched_topic])
                    else:
                        print(f"Low match score for topic '{topic}': {score}")
                else:
                    print(f"Invalid topic '{topic}'")
    elif isinstance(topics, str):
        topic_lower = topics.lower().strip()  # Convert to lowercase
        if is_valid_topic(topic_lower):
            topic_embedding = model.encode(topic_lower, convert_to_tensor=True)
            # Perform cosine similarity matching
            similarities = {key: util.pytorch_cos_sim(topic_embedding, emb).item() for key, emb in topic_embeddings.items()}
            matched_topic, score = max(similarities.items(), key=lambda x: x[1])
            if score >= threshold:
                topic_numbers.add(topic_mapping[matched_topic])
            else:
                print(f"Low match score for topic '{topics}': {score}")
        else:
            print(f"Invalid topic '{topics}'")

    # Remove None values and sort the topics
    topic_numbers = sorted([t for t in topic_numbers if t is not None])
    return topic_numbers

def preprocess_file(file_path):
    """
    Preprocess file with results to map the topics to the original topics list
    Writes the preprocessed file in a new file with the name 'file_path_processed'  in the folder 'results'
    Args:
        file_path (str): the file path to preprocess

    """
    processed_lines = []

    print(f"Processing file: {file_path}")

    with open(file_path, 'r', encoding='latin1') as file:
        lines = file.readlines()
        i = 0
        while i < len(lines):
            line = lines[i].strip()
            if line.lower().startswith("text:"):
                text_line = line
                print(f"Found text: {text_line}")
                i += 1
                while i < len(lines) and not lines[i].strip().lower().startswith("text:"):
                    line = lines[i].strip()
                    if re.search(r"\{'Topics':(.+?)\}", line):
                        match = re.search(r"\{'Topics':(.+?)\}", line)
                        topics_str = match.group(1).strip()
                    elif re.search(r"\['Topics':(.+?)\]", line):
                        match = re.search(r"\['Topics':(.+?)\]", line)
                        topics_str = match.group(1).strip()
                    else:
                        i += 1
                        continue

                    print(f"Found topics string: {topics_str}")

                    try:
                        topics = json.loads(topics_str.replace("'", "\""))
                        if isinstance(topics, str):
                            topics = [topics]
                        elif isinstance(topics, dict):
                            topics = [topics]
                    except json.JSONDecodeError:
                        topics = re.findall(r"'(.*?)'", topics_str) + re.findall(r"(\d+)", topics_str)

                    if not topics:
                        topics = topics_str.split(', ')

                    print(f"Extracted topics: {topics}")
                    processed_topics = process_topics(topics)
                    print(f"Processed topics: {processed_topics}")
                    processed_lines.append(f"{text_line}\n")
                    processed_lines.append(f"{{'Topics': {', '.join(map(str, processed_topics))}}}\n")
                    processed_lines.append("\n")
                    break
            elif line.strip().startswith("['Topics") or line.strip().startswith("Topics") or line.strip().startswith("'Topics"):
                topics_str = line.split(":", 1)[1].strip().strip('[]')
                print(f"Found topics string: {topics_str}")

                try:
                    topics = json.loads(topics_str.replace("'", "\""))
                    if isinstance(topics, str):
                        topics = [topics]
                    elif isinstance(topics, dict):
                        topics = [topics]
                except json.JSONDecodeError:
                    topics = re.findall(r"'(.*?)'", topics_str) + re.findall(r"(\d+)", topics_str)

                if not topics:
                    topics = topics_str.split(', ')

                print(f"Extracted topics: {topics}")
                processed_topics = process_topics(topics)
                print(f"Processed topics: {processed_topics}")
                processed_lines.append(f"{{'Topics': {', '.join(map(str, processed_topics))}}}\n")
                processed_lines.append("\n")
            i += 1
    # Save the processed file to the processed_data directory
    output_path = os.path.join(processed_data_dir, os.path.basename(file_path).replace(".txt", "_processed.txt"))
    with open(output_path, 'w', encoding='latin1') as output_file:
        output_file.writelines(processed_lines)
    print(f"Finished processing file: {file_path}")
    print(f"Processed file saved to: {output_path}\n")

# Ensure the processed_data directory exists
processed_data_dir = 'processed_data'
os.makedirs(processed_data_dir, exist_ok=True)

# Process each file
for file_path in file_paths_to_process:
    preprocess_file(file_path)

processed_file_paths = [os.path.join(processed_data_dir, os.path.basename(file_path).replace(".txt", "_processed.txt")) for file_path in file_paths_to_process]

print("Processing completed. Processed files are saved in the 'processed_data' folder.")


C:\Users\marcu\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Processing file: finetuning_results/finetuning_results_formatted.txt
Found text: Text: Residents are needed. It's also a lot of fun to get involved. However, there are quick choices of principle that exclude other solutions. Very valuable on a small scale (including mienskip) and that certainly in combination with climate and reuse. Local initiatives can also lead to fragmentation while electricity must always be available, and inefficiencies.
Found topics string: Municipality and residents engagement in the energy sector
Extracted topics: ['Municipality and residents engagement in the energy sector']
Processed topics: [1]
Found text: Text: I assign points to both "The municipality takes the lead and unburdens" and to "Residents do it themselves". This seems contradictory, and it is. Nevertheless, I also think the residents' own initiative is important. For support, for social cohesion, but also for management. A local cooperative will be able to supply energy significantly cheaper tha

In [2]:
# File that has been preprocessed
file_paths_processed = [
    f"processed_data/{fp.replace('finetuning_results/', '').replace('.txt', '_processed.txt')}"
    for fp in file_paths_to_process
]
print(file_paths_processed)
method = "Fine-tuning"

NameError: name 'file_paths_to_process' is not defined

In [12]:
def combine_results(file_paths):
    """
    Combine results from multiple files
    For each text -> topics from each file

    Args:
        file_paths (list): the list of file paths that has been preprocessed
    Returns:
        combined_results (list) : list combined results

    """
    combined_results = defaultdict(list)

    for file_path in file_paths:
        with open(file_path, 'r') as file:
            content = file.read().strip().split("\n\n")
            for entry in content:
                if 'Topics' in entry:
                    try:
                        text_part, label_part = entry.split("{'Topics': ")
                        text = text_part.replace("Text: ", "").strip()
                        topics = "{'Topics': " + label_part.strip()
                        combined_results[text].append(topics)
                    except ValueError:
                        print(f"Skipping entry due to parsing error: {entry}")

    return combined_results

def aggregate_results(combined_results):
    """
    Aggregate the topics results from multiple files using majority vote

    Args:
        combined_results (list): the list of combined results from preprocessed file (for each text -> topics from each file)
    Returns:
        aggregated_results (list) : list of aggregated results using majority vote

    """
    aggregated_results = {}

    for text, topics in combined_results.items():
        all_topics = [topic.strip() for topic_list in topics for topic in topic_list.replace("{'Topics': ", "").replace("}", "").split(',')]
        topic_counts = Counter(all_topics)
        majority_topics = [topic for topic, count in topic_counts.items() if count >= (len(files_to_format) / 2)]
        aggregated_results[text] = majority_topics

    return aggregated_results

def save_combined_results(combined_results, output_path):
    """
    Save in a new file the combined results from multiple files
    For each text -> topics from each file

    Args:
        combined_results (list): the list of combined results from preprocessed file (for each text -> topics from each file)
        output_path (str): the output path of the new file

    """
    with open(output_path, 'w') as file:
        for text, topics_list in combined_results.items():
            file.write(f"Text: {text}\n")
            for topics in topics_list:
                file.write(f"{topics}\n")
            file.write("\n")

def save_aggregated_results(aggregated_results, output_path):
    """
    Save the aggregated results

    Args:
        aggregated_results (list) : list of aggregated results using majority vote
        output_path (str): the output path of the new file
    """
    with open(output_path, 'w') as file:
        for text, topics in aggregated_results.items():
            file.write(f"Text: {text}\n")
            file.write(f"Assigned Topics: {', '.join(topics)}\n\n")

('aggregated_results/Combined_Results_Fine-tuning.txt',
 'aggregated_results/Aggregated_Results_Fine-tuning.txt')

In [ ]:
# Define the output paths for the combined and aggregated results text files
output_combined_text_path = 'aggregated_results/Combined_Results_' + method +'.txt'
output_aggregated_text_path = 'aggregated_results/Aggregated_Results_' + method +'.txt'

# Combine the results from the text files into the desired format
combined_results = combine_results(file_paths_processed)

# Save the combined results to a text file
save_combined_results(combined_results, output_combined_text_path)

# Aggregate the results using majority vote
aggregated_results = aggregate_results(combined_results)

# Save the aggregated results to a text file
save_aggregated_results(aggregated_results, output_aggregated_text_path)

(output_combined_text_path, output_aggregated_text_path)

### Final step for preprocessing results: saving the aggregated results in a csv file

In [3]:
# Empty csv file containing only the test data
empty_df = pd.read_csv('Test_finetuning_data.csv', delimiter=';')

# Define the mapping of labels to column names
label_to_column = {
    '1': 'Municipality and residents engagement in the energy sector',
    '2': 'Energy storage and supplying energy in The Netherlands',
    '3': 'Wind and solar energy',
    '4': 'Market Determination Dynamics',
    '5': 'Landscapes and windmills tourism',
    '6': 'Hydrogen energy pipeline networks'
}

# Initialize a dictionary to store text to index mapping
text_to_index = {}

# Map text to their row index in the empty CSV file
for idx, row in empty_df.iterrows():
    text = row['english'].strip()
    text_to_index[text] = idx

# Initialize all columns to '0'
for column in label_to_column.values():
    empty_df[column] = '0'

# Fill the columns based on the aggregated labels from the Aggregated_Results.txt file
output_aggregated_text_path = "aggregated_results/Aggregated_Results_"+ method + ".txt"
with open(output_aggregated_text_path, 'r') as file:
    content = file.read().strip().split("\n\n")
    for entry in content:
        lines = entry.split("\n")
        if len(lines) < 2:
            continue
        text = lines[0].replace("Text: ", "").strip()
        topics_line = lines[1].replace("Assigned Topics: ", "").strip()
        topics = topics_line.split(", ") if topics_line else []
        if text in text_to_index:
            idx = text_to_index[text]
            for label, column_name in label_to_column.items():
                if label in topics:
                    empty_df.at[idx, column_name] = '1'

# Save the filled CSV file
filled_csv_path_final = 'llm_annotated_data/Results_' + method +'.csv'
empty_df.to_csv(filled_csv_path_final, index=False, sep=';')

filled_csv_path_final


'llm_annotated_data/Results_Fine-tuning.csv'